In [8]:
import pandas as pd
import re
from datetime import datetime

parquet_file = r"C:/Users/rmartin10/Downloads/Challenge/configuration iptables/logiptables.parquet"
log_file = r"C:/Users/rmartin10/Downloads/Challenge/iptables_ulogd2.log"

log_pattern = re.compile(
    r'(?P<date>\w+ \d+ \d+:\d+:\d+) \S+ (?P<action>DENY|PERMIT)(?: RULE=(?P<rule>\d+))? .*?'
    r'IN=(?P<interface_in>\S+) OUT=(?P<interface_out>\S+)? .*?'
    r'SRC=(?P<ipsrc>\d+\.\d+\.\d+\.\d+) DST=(?P<ipdst>\d+\.\d+\.\d+\.\d+) .*?'
    r'PROTO=(?P<proto>\S+) SPT=(?P<portsrc>\d+) DPT=(?P<portdst>\d+)'
)

log_data = []

with open(log_file, "r") as f:
    for line in f:
        match = log_pattern.search(line)
        if match:
            log_entry = match.groupdict()
            log_entry["date"] = datetime.strptime(log_entry["date"], "%b %d %H:%M:%S").replace(year=2025)
            log_entry["portsrc"] = float(log_entry["portsrc"])
            log_entry["portdst"] = float(log_entry["portdst"])
            log_entry["rule"] = float(log_entry["rule"]) if log_entry["rule"] else 999.0
            log_entry["interface_out"] = log_entry["interface_out"] if log_entry["interface_out"] else "None"
            log_entry["divers"] = 6.0 

            log_data.append(log_entry)

df = pd.DataFrame(log_data)

df.to_parquet(parquet_file, engine="pyarrow")

print(f"Fichier Parquet créé : {parquet_file}")

Fichier Parquet créé : C:/Users/rmartin10/Downloads/Challenge/configuration iptables/logiptables.parquet
